In [1]:
# !pip install -r requirements.txt
import importlib
importlib.reload
# Load the TensorBoard notebook extension
%load_ext tensorboard
import torch, os, sys
print("torch version  :", torch.__version__)

torch version  : 2.9.0.dev20250715


In [2]:
!python scripts/convert_dm4.py --input data/Diffraction_SI.dm4 --output data/train_tensor.h5 --downsample 16 --mode bin 

python: can't open file '/Users/louisg/PycharmProjects/4DSTEM_AE_MSc/scripts/convert_dm4.py': [Errno 2] No such file or directory


In [3]:
!python -m scripts.train --data data/train_tensor.pt --epochs 1 --batch 32 --latent 8 --lr 0.00003 --output_dir outputs --device mps --summary True

/Users/louisg/.pyenv/versions/custom_ae/bin/python: No module named scripts.train


In [4]:
!python -m scripts.reconstruct --checkpoint outputs/ae.ckpt --data data/train_tensor_ds1.pt --num_samples 32

/Users/louisg/.pyenv/versions/custom_ae/bin/python: No module named scripts.reconstruct


In [5]:
!python -m scripts.generate_embeddings --input data/train_tensor.pt --checkpoint outputs/ae.ckpt --batch_size 2048 --output outputs/embeddings.pt

/Users/louisg/.pyenv/versions/custom_ae/bin/python: No module named scripts.generate_embeddings


In [6]:
import torch, numpy as np
raw = torch.load("data/train_tensor.pt")
print("raw tensor shape:", raw.shape)         # (N, 1, Qy, Qx)
N = raw.shape[0]
print("number of probe positions:", N)

# quick factor search
cands = [(f, N//f) for f in range(1, int(np.sqrt(N))+1) if N % f == 0]
print("factor pairs:", cands[:10], "…")


raw tensor shape: torch.Size([4788, 1, 32, 32])
number of probe positions: 4788
factor pairs: [(1, 4788), (2, 2394), (3, 1596), (4, 1197), (6, 798), (7, 684), (9, 532), (12, 399), (14, 342), (18, 266)] …


In [ ]:
python scripts/visualization/visualize_h5_dataset.py workspace/ds4_test_input_data/inputs/train_tensor_ds4_compressed.h5 --output workspace/diagrams/model_arch/stem_visualisations/stem_vis_eda.png --interactive-center

python: can't open file '/Users/louisg/PycharmProjects/4DSTEM_AE_MSc/scripts/stem_visualization.py': [Errno 2] No such file or directory


In [8]:
!python -m scripts.evaluate_autoencoder --checkpoint outputs/ae.ckpt --data_path data/train_tensor.pt --output_path outputs/eval/evaluation_report.png

/Users/louisg/.pyenv/versions/custom_ae/bin/python: No module named scripts.evaluate_autoencoder


In [ ]:
python scripts/visualization/visualise_scan_latents.py --raw workspace/proj_data_ds4_l8/inputs/train_tensor_ds4_compressed.h5 --latents workspace/proj_data_ds4_l16/outputs/embeddings/ds4_4epoch_embeddings.npz --scan 209 194 --lat_max_cols 8 --outfig workspace/proj_data_ds4_l16/visualisations

SyntaxError: invalid syntax (4260610554.py, line 1)

## Dimension Reduction

In [ ]:
python scripts/analysis/umap_latent_visualization.py --embeddings workspace/proj_data_ds4_l32_with_reg/dimension_reduction/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering --n_neighbors 50 --min_dist 0.01

In [ ]:
python scripts/analysis/cluster_scan_visualization.py --umap_results "workspace/proj_data_ds4_l32_with_reg/clustering/analysis_new/clustering_analysis_data.npz" --raw_data "workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5" --output_dir "workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_md_pointzero1/stem_vis_overlay" --scan_shape 209 194 

In [ ]:
#  dimension reduction with UMAP
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l32_with_reg/embeddings/ds4_32lat_with_regs_embeddings.pt --method umap --optimize_parameters --subsample 5000

In [ ]:
# pca optimisation
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir "workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/pca_optimisation" --method pca --pca_component_sweep "3,4,5,6,7,8" --optimize_parameters --standardize --scan_shape 209 194
# umap optimisation
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir "workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation" --method umap --optimize_parameters --standardize --scan_shape 209 194


In [ ]:
# PCA with single component count (no optimization)
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l32_with_reg/embeddings/ds4_32lat_with_regs_embeddings.pt --output_dir workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components --method pca --n_components 15 --standardize --scan_shape 209 194

## Clustering

In [ ]:
  # For original latent embeddings (high-dimensional)
  python scripts/analysis/optimize_clustering.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir "workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_optimisation" --max_clusters 20 --subsample 10000 --standardize

In [ ]:
  # For UMAP/PCA reduced embeddings  
  python scripts/analysis/optimize_clustering.py \
      --reduced_data results/dimension_reduction/umap_reduction_data.npz \
      --output_dir results/reduced_clustering_optimization \
      --max_clusters 15

In [ ]:
python scripts/analysis/clustering_analysis.py --reduced_embeddings workspace/proj_data_ds4_l32_with_reg/dimension_reduction_optimisation/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering/analysis --method kmeans --n_clusters 4 --standardize

In [ ]:
 # Compare multiple clustering methods for latent embeddings directly from ae
    python scripts/analysis/clustering_analysis.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir workspace/proj_data_ds4_l32/visualisations/latent_embeddings_cluster_analysis --method all --compare_methods

In [ ]:
python scripts/analysis/optimize_clustering.py --reduced_data workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation/umap_reduction_data.npz  --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation/clustering_optimisation --clusters "3"

In [ ]:
# Clustering analysis on original latent embeddings k-means
    python scripts/analysis/clustering_analysis.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/latent_clustering --method kmeans --n_clusters 3 --standardize

In [ ]:
# Clustering analysis on reduced embeddings k-means
    python scripts/analysis/clustering_analysis.py --reduced_embeddings workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_k_7/umap_data.npz --output_dir workspace/proj_data_ds4_l32/visualisations/clustering_analysis_umap_k_3 --method kmeans --n_clusters 3 --standardize

In [ ]:
python scripts/analysis/optimize_clustering.py --reduced_data workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components/pca_reduction_data.npz  --output_dir workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components/clustering_optimisation --max_clusters 7 --subsample 10000

SyntaxError: invalid syntax (3418114949.py, line 1)

In [ ]:
python scripts/analysis/optimize_clustering.py --umap_data workspace/proj_data_ds4_l32_with_reg/dimension_reduction_single/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering --max_clusters 20 --subsample 10000

# latents clustering
python scripts/analysis/optimize_clustering.py --latent_embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/latent_clustering --clusters "3" --standardize

## Overlay Visualisation

In [ ]:
 python scripts/analysis/cluster_scan_visualiza# scan, bright‑field background, every latent dim in one mosaic
python scripts/visualization/visualise_scan_latents.py --raw workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --latents workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --scan 194 209 --virtual bf --lat_max_cols 6 --outfig workspace/proj_data_ds4_l32/outputs/latent_mosaic.pngtion.py --umap_results workspace/proj_data_ds4_l32/visualisations/spectral_clustering_optimisation/spectral_individual_results/spectral_3/cluster_info_spectral_3_clus.json --scan_shape 209 194 --output_dir workspace/proj_data_ds4_l32/visualisations/spectral_clustering_optimisation/stem_vis_overlay --raw_data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5

In [ ]:
python scripts/analysis/cluster_scan_visualization.py --clustered_data workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy  --scan_shape 209 194 --output_dir workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/stem_vis_overlay --raw_data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5

In [ ]:
python scripts/analysis/umap_latent_visualization.py \
        --embeddings embeddings/patterns_embeddings.npz \
        --output_dir results/umap_analysis \
        --n_neighbors 50 \
        --min_dist 0.01

## Polar Mapping

In [ ]:
python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_k_7/umap_data.npz --scan-shape 209 194 --config workspace/proj_data_ds4_l32/visualisations/polar_map_improved/improved_polar_config.yaml --out workspace/proj_data_ds4_l32/visualisations/polar_map_improved

In [ ]:

python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy --scan-shape 209 194 --config workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map/polar_config.yaml --out workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map --interactive-ks
pace

In [ ]:
python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy --scan-shape 209 194 --config workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map/polar_config.yaml --out workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map

In [ ]:
python -m scripts.visualization.diagrams.visualize_with_torchlens --model models.autoencoder:Autoencoder --scan-y 209 --scan-x 194 --det 256 --latent-dim 32 --out torchlens_viz

In [ ]:
# Control nesting depths and which blocks to show:
  python -m scripts.visualization.diagrams.visualize_with_torchlens --model models.autoencoder:Autoencoder --blocks-depth 1 --inside-depth 4 --focus-modules encoder.resnet1,decoder.resnet_up1 --out torchlens_viz

In [ ]:
dot -Tsvg workspace/diagrams/landscape/torchlens_final_inside_decoder_decoder_resnet_up1_clean_landscape.dot -o torchlens_final_inside_decoder_decoder_resnet_up1_clean_landscape.svg


## CoM analysis

In [ ]:
python scripts/analysis/com_polarization_map.py workspace/proj_data_ds4_l8/inputs/train_tensor_ds4_compressed.h5 --datapath patterns --scan-shape 209 194 --beamstop --mask-radius 8 --bin-q 2 --rotate 0 --outdir workspace/proj_data_ds4_l8/outputs/com_maps 

In [ ]:
python scripts/analysis/latent_cluster_polar_map.py --mode conventional --data workspace/proj_data_ds4_l8/inputs/train_tensor_ds4_compressed.h5 --dset patterns --scan-shape 209 194 --out workspace/proj_data_ds4_l8/outputs/conventional_dpc --chunks 512 --device cpu --config workspace/proj_data_ds4_l8/outputs/com_maps/polar_config.yaml